### Aim

Ok so We are going to use the **INDRA Person Dataset** to create the negatives for this project. 

So the Dataset has the structure:
```
.
├── Test
│   ├── Annotations
│   └── JPEGImages
└── Train
    ├── Annotations
    └── JPEGImages

```

Where Annotations contains an .XML file outlineing where the pedestrians are in the picture. 

The way we plan to make images without pedestrians for our dataset is to scan through these images and take portions of the image that dont include objects. 

### Execution

In this notebook, we utilise the INDRA Person Images dataset to validate person object annotations. For each annotated person (represented by a bounding box), we examine the adjacent regions directly to the left and right of the box. If these neighbouring regions do not contain any annotated persons, we extract them and resize to 64x128 pixels to create candidate ‘negative’ samples—images that are presumed to contain no people.

However, because the original dataset annotations are not perfectly accurate, some of these regions may still include people. To address this, we manually review and select negative samples to ensure they do not contain any visible persons.

##### Left Right Reasoning

We intentionally restrict our negative sampling to areas horizontally adjacent (left and right) to detected persons. This is based on the assumption that people appear on the ground plane. By using existing person annotations as a reference for the horizontal level and scale where people are likely to appear, we can more reliably sample background regions that are at a similar height and perspective, improving the consistency of our negative examples.

### Setup

Lets first get the images and annotations needed

In [23]:
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path

def parse_annotation(annotation_path):
    """Parse XML annotation file and extract person bounding boxes."""
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    
    # Get image dimensions
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    
    # Extract all person bounding boxes
    bboxes = []
    for obj in root.findall('object'):
        if obj.find('name').text == 'person':
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)
            bboxes.append((xmin, ymin, xmax, ymax))
    
    print(f"Parsing annotation: {annotation_path}")
    print(f"Image size: {width}x{height}")
    print(f"Found {len(bboxes)} person bounding boxes: {bboxes}")
    return width, height, bboxes

def region_overlaps_with_person(region, person_boxes, margin=0):
    """Check if a region overlaps with any person bounding box."""
    x1, y1, x2, y2 = region
    
    for box_x1, box_y1, box_x2, box_y2 in person_boxes:
        # Add margin around person boxes
        box_x1 -= margin
        box_y1 -= margin
        box_x2 += margin
        box_y2 += margin
        
        # Check if there's any overlap
        if not (x2 < box_x1 or x1 > box_x2 or y2 < box_y1 or y1 > box_y2):
            print(f"Region {region} overlaps with person box {(box_x1, box_y1, box_x2, box_y2)}")
            return True
    
    return False

def extract_left_right_negatives(img, width, height, bboxes, margin=5):
    negative_regions = []
    for i, (x1, y1, x2, y2) in enumerate(bboxes):
        person_width = x2 - x1
        person_height = y2 - y1
        print(f"Person {i+1}: bbox=({x1},{y1},{x2},{y2}), size={person_width}x{person_height}")
        
        # Try left region - use same dimensions as person box
        left_x1 = x1 - person_width - margin
        left_x2 = x1 - margin
        left_y1 = y1  # Use same y-coordinates as person
        left_y2 = y2
        print(f"  Trying left region: ({left_x1},{left_y1},{left_x2},{left_y2})")
        if left_x1 >= 0:
            left_region = (left_x1, left_y1, left_x2, left_y2)
            if not region_overlaps_with_person(left_region, bboxes):
                print(f"    Accepted left region: {left_region}")
                negative_regions.append(left_region)
            else:
                print(f"    Rejected left region due to overlap")
        else:
            print(f"    Skipping left region: out of bounds (left_x1={left_x1} < 0)")
            
        # Try right region - use same dimensions as person box
        right_x1 = x2 + margin
        right_x2 = x2 + person_width + margin
        right_y1 = y1  # Use same y-coordinates as person
        right_y2 = y2
        print(f"  Trying right region: ({right_x1},{right_y1},{right_x2},{right_y2})")
        if right_x2 <= width:
            right_region = (right_x1, right_y1, right_x2, right_y2)
            if not region_overlaps_with_person(right_region, bboxes):
                print(f"    Accepted right region: {right_region}")
                negative_regions.append(right_region)
            else:
                print(f"    Rejected right region due to overlap")
        else:
            print(f"    Skipping right region: out of bounds (right_x2={right_x2} > width={width})")
    print(f"Total negative regions found: {len(negative_regions)}")
    return negative_regions

def process_single_image(image_path, annotation_path, output_dir):
    """
    Process a single image and its annotation file to extract negative samples.
    
    Args:
        image_path: Path to the image file
        annotation_path: Path to the XML annotation file
        output_dir: Directory to save negative samples
    
    Returns:
        num_negatives: Number of negative samples extracted
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Parse annotation
    try:
        width, height, bboxes = parse_annotation(annotation_path)
    except Exception as e:
        print(f"Error parsing annotation {annotation_path}: {e}")
        return 0
    
    # Load image
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Error: Could not read image {image_path}")
        return 0
    
    # Extract negative regions
    negative_regions = extract_left_right_negatives(img, width, height, bboxes)
    
    # Save negative samples
    base_name = os.path.splitext(os.path.basename(str(image_path)))[0]
    
    for i, (x1, y1, x2, y2) in enumerate(negative_regions):
        negative_patch = img[y1:y2, x1:x2]
        
        # Resize to 64x128
        negative_patch = cv2.resize(negative_patch, (64, 128))
        
        if (i % 2) == 0:
            negative_filename = f"{base_name}_L{(i // 2) + 1}.jpg"
        elif (i % 2) == 1:
            negative_filename = f"{base_name}_R{(i // 2) + 1}.jpg"
        else:
            negative_filename = f"{base_name}_{i}.jpg"
        negative_path = os.path.join(str(output_dir), negative_filename)
        
        # Save the negative patch
        cv2.imwrite(negative_path, negative_patch)
    
    print(f"Extracted {len(negative_regions)} negative samples from {os.path.basename(str(image_path))}")
    return len(negative_regions)

def process_single_image_by_name(image_name, dataset_root, split='Train', output_dir='INRIA_Negatives'):
    """
    Process a single image by its name, automatically finding annotation and image paths.
    
    Args:
        image_name: Base name of the image (e.g., 'crop_001009' without extension)
        dataset_root: Root directory of the INRIA dataset
        split: 'Train' or 'Test'
        output_dir: Directory to save negative samples
    
    Returns:
        num_negatives: Number of negative samples extracted
    """
    # Create paths
    dataset_root = Path(dataset_root)
    output_dir = Path(output_dir)
    
    # Try to find the image with various extensions
    image_path = None
    for ext in ['.png', '.jpg', '.jpeg']:
        potential_path = dataset_root / split / 'JPEGImages' / f"{image_name}{ext}"
        if potential_path.exists():
            image_path = potential_path
            break
    
    if image_path is None:
        print(f"Error: Could not find image with name {image_name} in {split} set")
        return 0
    
    # Find annotation file
    annotation_path = dataset_root / split / 'Annotations' / f"{image_name}.xml"
    
    if not annotation_path.exists():
        print(f"Error: Could not find annotation file for {image_name}")
        return 0
    
    # Process the image
    return process_single_image(image_path, annotation_path, output_dir)

def process_all_images(dataset_root, output_dir):
    """
    Process all images in the INRIA dataset to extract negative samples.
    
    Args:
        dataset_root: Path to the INRIA dataset root
        output_dir: Directory to save negative samples
    """
    dataset_root = Path(dataset_root)
    output_dir = Path(output_dir)
    total_negatives = 0
    
    # Process both train and test sets
    for split in ['Train', 'Test']:
        annotations_dir = dataset_root / split / 'Annotations'
        
        annotation_files = list(annotations_dir.glob('*.xml'))
        print(f"Processing {len(annotation_files)} files in {split} set...")
        
        for annotation_file in tqdm(annotation_files):
            # Get image name (without extension)
            image_name = annotation_file.stem
            
            # Process this single image
            num_negatives = process_single_image_by_name(
                image_name, dataset_root, split, output_dir
            )
            total_negatives += num_negatives
    
    print(f"Extracted a total of {total_negatives} negative samples")

if __name__ == "__main__":
    # Get the current script's directory
    current_dir = Path.cwd()
    
    # Define the dataset root relative to the current directory
    dataset_root = current_dir / "INRIAPerson"
    output_dir = current_dir / "INRIA_Negatives"
    
    # Ensure output directory exists
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Process a single image by name
    image_name = "crop001046"  # Just the base name without extension
    split = "Train"  # 'Train' or 'Test'
    
    print(f"Processing {image_name} from {split} set...")
    num_negatives = process_single_image_by_name(
        image_name, dataset_root, split, output_dir
    )
    print(f"Extracted {num_negatives} negative samples from {image_name}")
    

Processing crop001046 from Train set...
Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Research Proj/HOG-for-Human-Detection-main- v0/Laines Work/INRIAPerson/Train/Annotations/crop001046.xml
Image size: 439x478
Found 1 person bounding boxes: [(163, 137, 257, 350)]
Person 1: bbox=(163,137,257,350), size=94x213
  Trying left region: (64,137,158,350)
    Accepted left region: (64, 137, 158, 350)
  Trying right region: (262,137,356,350)
    Accepted right region: (262, 137, 356, 350)
Total negative regions found: 2
Extracted 2 negative samples from crop001046.png
Extracted 2 negative samples from crop001046


Sweet, that works for 1 image, now lets run it for all images in our INRIAPerson Test and Train databases.  

In [24]:
if __name__ == "__main__":
    # Get the current script's directory
    current_dir = Path.cwd()
    
    # Define the dataset root relative to the current directory
    dataset_root = current_dir / "INRIAPerson"
    output_dir = current_dir / "INRIA_Negatives"
    
    # Ensure output directory exists
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Process all images in the dataset
    process_all_images(dataset_root, output_dir)
    print(f"All images processed. Negative samples saved in {output_dir}")

Processing 614 files in Train set...


  0%|          | 0/614 [00:00<?, ?it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
  2%|▏         | 12/614 [00:00<00:05, 117.89it/s]

Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Research Proj/HOG-for-Human-Detection-main- v0/Laines Work/INRIAPerson/Train/Annotations/person_121.xml
Image size: 480x640
Found 2 person bounding boxes: [(189, 139, 294, 473), (149, 112, 180, 218)]
Person 1: bbox=(189,139,294,473), size=105x334
  Trying left region: (79,139,184,473)
Region (79, 139, 184, 473) overlaps with person box (149, 112, 180, 218)
    Rejected left region due to overlap
  Trying right region: (299,139,404,473)
    Accepted right region: (299, 139, 404, 473)
Person 2: bbox=(149,112,180,218), size=31x106
  Trying left region: (113,112,144,218)
    Accepted left region: (113, 112, 144, 218)
  Trying right region: (185,112,216,218)
Region (185, 112, 216, 218) overlaps with person box (189, 139, 294, 473)
    Rejected right region due to overlap
Total negative regions found: 2
Extracted 2 negative samples from person_121.png
Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Rese

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
  6%|▌         | 36/614 [00:00<00:05, 109.74it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Person 1: bbox=(291,138,382,396), size=91x258
  Trying left region: (195,138,286,396)
Region (195, 138, 286, 396) overlaps with person box (197, 141, 251, 322)
    Rejected left region due to overlap
  Trying right region: (387,138,478,396)
    Accepted right region: (387, 138, 478, 396)
Person 2: bbox=(197,141,251,322), size=54x181
  Trying left region: (138,141,192,322)
    Accepted left region: (138, 141, 192, 322)
  Trying right region: (256,141,310,322)
Region (256, 141, 310, 322) overlaps with person box (291, 138, 382, 396)
    Rejected right region due to overlap
Total negative regions found: 2
Extracted 2 negative samples from person_281.png
Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Research Proj/HOG-for-Human-Detection-main- v0/Laines Work/INRIAPerson/Train/Annotations/person_256.xml
Image size: 640x480
Found 1 person bounding boxes: [(321, 100, 443, 432)]
Person 1: bbox=(321,100,443,432), size=122x332
  Trying left region: (194,100,316,432)
    Acc

...

Person 1: bbox=(299,31,434,418), size=135x387
  Trying left region: (159,31,294,418)
    Accepted left region: (159, 31, 294, 418)
  Trying right region: (439,31,574,418)
    Accepted right region: (439, 31, 574, 418)
Total negative regions found: 2
Extracted 2 negative samples from person_and_bike_082.png
Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Research Proj/HOG-for-Human-Detection-main- v0/Laines Work/INRIAPerson/Test/Annotations/person_and_bike_096.xml
Image size: 640x480
Found 1 person bounding boxes: [(201, 41, 335, 388)]
Person 1: bbox=(201,41,335,388), size=134x347
  Trying left region: (62,41,196,388)
    Accepted left region: (62, 41, 196, 388)
  Trying right region: (340,41,474,388)
    Accepted right region: (340, 41, 474, 388)
Total negative regions found: 2
Extracted 2 negative samples from person_and_bike_096.png
Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Research Proj/HOG-for-Human-Detection-main- v0/Laines Work/INRI

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
 95%|█████████▌| 275/288 [00:02<00:00, 115.48it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Person 1: bbox=(533,248,628,582), size=95x334
  Trying left region: (433,248,528,582)
    Accepted left region: (433, 248, 528, 582)
  Trying right region: (633,248,728,582)
    Accepted right region: (633, 248, 728, 582)
Total negative regions found: 2
Extracted 2 negative samples from crop001549.png
Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Research Proj/HOG-for-Human-Detection-main- v0/Laines Work/INRIAPerson/Test/Annotations/person_065.xml
Image size: 640x480
Found 1 person bounding boxes: [(261, 125, 352, 456)]
Person 1: bbox=(261,125,352,456), size=91x331
  Trying left region: (165,125,256,456)
    Accepted left region: (165, 125, 256, 456)
  Trying right region: (357,125,448,456)
    Accepted right region: (357, 125, 448, 456)
Total negative regions found: 2
Extracted 2 negative samples from person_065.png
Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Research Proj/HOG-for-Human-Detection-main- v0/Laines Work/INRIAPerson/Test/Ann

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
100%|██████████| 288/288 [00:02<00:00, 109.58it/s]

Person 1: bbox=(203,212,296,487), size=93x275
  Trying left region: (105,212,198,487)
    Accepted left region: (105, 212, 198, 487)
  Trying right region: (301,212,394,487)
    Accepted right region: (301, 212, 394, 487)
Total negative regions found: 2
Extracted 2 negative samples from crop001574.png
Parsing annotation: /Users/lainemulvay/Desktop/Projects/UNI/cits4402/Research Proj/HOG-for-Human-Detection-main- v0/Laines Work/INRIAPerson/Test/Annotations/person_306.xml
Image size: 640x480
Found 3 person bounding boxes: [(254, 147, 358, 428), (155, 136, 252, 457), (424, 165, 456, 256)]
Person 1: bbox=(254,147,358,428), size=104x281
  Trying left region: (145,147,249,428)
Region (145, 147, 249, 428) overlaps with person box (155, 136, 252, 457)
    Rejected left region due to overlap
  Trying right region: (363,147,467,428)
Region (363, 147, 467, 428) overlaps with person box (424, 165, 456, 256)
    Rejected right region due to overlap
Person 2: bbox=(155,136,252,457), size=97x321
  Tr

This does a pretty good job but objects (people) aren't all labelled perfectly in the INRIAPerson Dataset. Therefore we will **maunally select images** for negatives from this set